# Mappings Transformation

Convert the Emacs Lisp definition of each mapping into a C# Dictionary

In [ ]:
import re

In [ ]:
# load the raw data
# This file was created using `agda-input-show-translations`, with the terminal
#  small enough that only a single line could be displayed. this was then saved
#  to a file
with open("../assets/mappings.txt", "r", encoding="utf-8") as f:
    _, single_chars, multi_chars = f.read().split("-\n")

In [ ]:
# single char matches
single_matcher = re.compile(r"\\(.*?)\s+(.{1,4})(?:$|\n)")
single_char_replacements = single_matcher.findall(single_chars)
single_char_replacements

In [ ]:
multi_matcher = re.compile(r"\\(.*?)(?:\s+)((?:.|\n\s+)*)")
multi_multiline_cleanup = re.compile(r"\n\s+")
multi_char_replacements = multi_matcher.findall(multi_chars)
multi_char_replacements = [
    (k, multi_multiline_cleanup.sub(" ", v)) for k, v in multi_char_replacements
]
multi_char_replacements

In [ ]:
lookup = {
    k.replace("\\", "\\\\").replace("\"", "\\\""): v
    for k, v in (single_char_replacements + multi_char_replacements)
}
PROBLEMATIC_KEYS = ("", "-", "newline")
for key in PROBLEMATIC_KEYS:
    lookup.pop(key)
lookup

In [ ]:
# test querying capability
query_matcher = re.compile(r"^(.*?)(\d*)$")

def query(i):
    key, index = query_matcher.findall(i)[0]
    if index:
        index = 2 * int(index) - 1
    else:
        index = 1

    if key in lookup:
        return lookup[key][index - 1]
    else:
        raise ValueError

In [ ]:
query(":4")

In [ ]:
# export dictionary to Lookup.cs
with open("Lookup.template.cs", "r", encoding="utf-8") as template:
    with open("../src/nthn.Agda/Lookup.cs", "w", encoding="utf-8") as output:
        output.write(
"""// README: this file was generated from `Lookup.template.cs` in `tools/`. Make sure you apply any edits
//         you make to this file to that as well so the edits can be replicated if you regenerate this
//         file!
"""
        )
        template_lines = template.readlines()
        for line in template_lines[:9]:
            output.write(line)
        
        total = len(lookup.keys())
        for i, (k, v) in enumerate(lookup.items()):
            output.write(f"            {{\"{k}\", \"{v}\"}}")
            if i == total - 1:
                output.write("\n")
            else:
                output.write(",\n")
        
        for line in template_lines[9:]:
            output.write(line)